In [1]:
import glob
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip

%matplotlib inline

-----------------------
None
ffmpeg -version
-----------------------


In [2]:
from src.calibrator import Calibrator
from src.edges import get_mask
from src.birdView import BirdView
from src.lineFinder import LineFinder
from src.measurements import Measurements
from src.utils import show_info

In [3]:
Calibrator.calibrate()

Calibrator :: Calibration performed successfully


In [4]:
left_line = None
right_line = None

def pipeline(image):
    undistorted_image = Calibrator.undistort_image(image)
    transformed_image = BirdView.transform_perspective(undistorted_image)
    mask = get_mask(transformed_image)
    
    global left_line, right_line
    
    (left_line, right_line, left_fitx, right_fitx, ploty, result_img) = LineFinder.find_lines(
        mask,
        left_line,
        right_line
    )

    left_curvature, right_curvature = Measurements.get_curvature(ploty, left_fitx, right_fitx)
    shift = Measurements.get_shift(left_line, right_line)
    
    annotated_image = show_info(
        undistorted_image, mask, left_fitx, right_fitx, ploty, left_curvature, right_curvature, shift
    )

    return annotated_image

In [5]:
clip_name = VideoFileClip('project_video.mp4')
output_name = 'result_video.mp4'

output = clip_name.fl_image(pipeline)
%time output.write_videofile(output_name, audio=False)

-----------------------
None
C:\Users\Artyom\AppData\Local\imageio\ffmpeg\ffmpeg.win32.exe -i project_video.mp4
-----------------------
-----------------------
None
C:\Users\Artyom\AppData\Local\imageio\ffmpeg\ffmpeg.win32.exe -i project_video.mp4 -loglevel error -f image2pipe -vf scale=1280:720 -sws_flags bicubic -pix_fmt rgb24 -vcodec rawvideo -
-----------------------
-----------------------
None
C:\Users\Artyom\AppData\Local\imageio\ffmpeg\ffmpeg.win32.exe -i project_video.mp4
-----------------------
-----------------------
None
C:\Users\Artyom\AppData\Local\imageio\ffmpeg\ffmpeg.win32.exe -i project_video.mp4 -vn -loglevel error -f s16le -acodec pcm_s16le -ar 44100 -ac 2 -
-----------------------
[MoviePy] >>>> Building video result_video.mp4
[MoviePy] Writing video result_video.mp4
-----------------------
None
C:\Users\Artyom\AppData\Local\imageio\ffmpeg\ffmpeg.win32.exe -y -loglevel error -f rawvideo -vcodec rawvideo -s 1280x720 -pix_fmt rgb24 -r 25.00 -i - -an -vcodec libx264 -

100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [03:38<00:00,  5.89it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result_video.mp4 

Wall time: 3min 38s
